In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams["figure.figsize"] = (15,10)
%matplotlib inline

from tqdm import tqdm_notebook #show iter progression

Llegim els `.csv` reduïts de cada _scraping_. Es troben dins la següent estructura de directoris: 
```bash
.
├── 02102015
│   ├── 02102015.csv
│   └── listings.csv
├── 03012016
│   ├── 03012016.csv
│   └── listings.csv
├── 08042017
│   ├── 08042017.csv
│   └── listings.csv
├── 08122016
│   ├── 08122016.csv
│   └── listings.csv
├── 17072015
│   ├── 17072015.csv
│   └── listings.csv
└── 30042015
    ├── 30042015.csv
    └── listings.csv
```

on "`listing.csv`" és l'arxiu original de descarregar d'inside airbnb i "`[0-9].csv`" és l'arxiu reduït amb una subselecció de columnes

In [2]:
df1 = pd.read_csv('listings/30042015/30042015.csv', sep = ";")
df2 = pd.read_csv('listings/17072015/17072015.csv', sep = ";")
df3 = pd.read_csv('listings/02102015/02102015.csv', sep = ";")
df4 = pd.read_csv('listings/03012016/03012016.csv', sep = ";")
df5 = pd.read_csv('listings/08122016/08122016.csv', sep = ";")
df6 = pd.read_csv('listings/08042017/08042017.csv', sep = ";")

In [3]:
dfs_l = (df1, df2, df3, df4, df5, df6) 

#convertim a datime per cada df
for df in dfs_l:
    df.host_since = pd.to_datetime(df.host_since, format="%Y-%m-%d")
    df.last_scraped = pd.to_datetime(df.last_scraped, format="%Y-%m-%d")    

feim un DataFrame on cada columna conté els `host_id` de cada _scrap_ i de nom li posam la data de l'_scrap_

In [4]:
l_hosts = [df['host_id'].values for df in dfs_l]  
df_hosts = pd.DataFrame(l_hosts)
df_hosts = df_hosts.T

In [5]:
df_hosts.columns = ['2015-04-30','2015-07-17','2015-10-02','2016-01-03','2016-12-08','2017-04-08']
df_hosts = df_hosts.apply(lambda x: x.sort_values().values)
print ([len(x) for x in l_hosts])
df_hosts.head()

[12033, 14703, 14539, 14855, 17369, 17653]


,2015-04-30,2015-07-17,2015-10-02,2016-01-03,2016-12-08,2017-04-08
0,346.0,3073.0,3073.0,3073.0,10704.0,10704.0
1,10704.0,3073.0,3073.0,3073.0,10704.0,10704.0
2,10704.0,10587.0,3073.0,10704.0,10704.0,10704.0
3,10704.0,10704.0,10587.0,10704.0,10704.0,10704.0
4,10704.0,10704.0,10704.0,10704.0,10704.0,10704.0


Feim un dataframe amb l'índex dels IDs únics de tots els dataframes i hi afegim els valors de les altres llistes a la posició corresponent, deixant espais buits on no s'ha trobat el `host_id`


In [6]:
uniq_id=np.sort(np.unique(np.hstack(l_hosts)))

id_df = pd.DataFrame(uniq_id)
id_df.set_index(0, inplace=True)

In [ ]:
#molt millorable
## Ignasi no miris :/

for date in tqdm_notebook(df_hosts.columns): 
    id_df[date]=''
    for i in tqdm_notebook(id_df.index):
        if  np.any(df_hosts[date].isin([i])):
            id_df[date].loc[i] = i
        else:
            id_df[date].loc[i] = np.nan

In [ ]:
id_df.head()

L'última entrada vàlida de cada fila ens dirà quin va ser el derrer cop que aquell host va ser vist en un _scrap_:

In [ ]:
last_seen = id_df.apply(lambda x: x.last_valid_index(), axis=1) #magic function last_valid_index!
last_seen = pd.DataFrame(last_seen, columns=['host_until'])
last_seen.host_until = pd.to_datetime(last_seen.host_until, format="%Y-%m-%d") 
last_seen_dict = pd.Series(last_seen, index = last_seen.index).to_dict()

In [ ]:
#mapejam el valor de l'ultima entrada valida al host_id per obtenir "host_until"
listing_tot = pd.concat(dfs_l)
listing_tot['host_until'] = listing_tot.host_id.map(last_seen_dict)
listing_tot.head()

In [ ]:
listing_tot.to_csv('listings_host_until.csv',sep=';', index=False)